<a href="https://colab.research.google.com/github/KaggleStudy4th/4th_kaggle_study/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/6%EC%A3%BC%EC%B0%A8/dacon_2_kiy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd /content/drive/MyDrive/dacon/물류 유통량 예측 경진대회.zip

!unzip -qq "/content/drive/MyDrive/dacon/물류 유통량 예측 경진대회.zip"

[Errno 20] Not a directory: '/content/drive/MyDrive/dacon/물류 유통량 예측 경진대회.zip'
/content
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
data=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         31684 non-null  int64 
 1   송하인_격자공간고유번호  31684 non-null  int64 
 2   수하인_격자공간고유번호  31684 non-null  int64 
 3   물품_카테고리       31684 non-null  object
 4   운송장_건수        31684 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ MB


In [15]:
data.head(3)

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3


#### 결측치 없음, 문자열 변수 1개

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         7920 non-null   int64 
 1   송하인_격자공간고유번호  7920 non-null   int64 
 2   수하인_격자공간고유번호  7920 non-null   int64 
 3   물품_카테고리       7920 non-null   object
dtypes: int64(3), object(1)
memory usage: 247.6+ KB


# 데이터 전처리

In [25]:
#문자열 데이터 처리
from sklearn.preprocessing import LabelEncoder
cols = data.columns[data.dtypes == object]

for col in cols:
  encoder = LabelEncoder()
  encoder.fit(np.concatenate((data[col], test[col]))) #train, test 합쳐서 인코딩

  data[col] = encoder.transform(data[col])
  test[col] = encoder.transform(test[col])

In [26]:
#가장 기본적인 통계치를 확인하기 위한 단계
train=data
train.describe()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
count,31684.000000,3.168400e+04,3.168400e+04,31684.000000,31684.000000
mean,15841.500000,4.648814e+15,3.801369e+15,31.810409,4.879308
std,9146.527301,8.341336e+14,1.314660e+15,16.899420,6.756473
min,0.000000,1.111000e+15,1.111000e+15,0.000000,3.000000
25%,7920.750000,4.713000e+15,2.826000e+15,27.000000,3.000000
50%,15841.500000,5.011000e+15,4.159000e+15,27.000000,3.000000
75%,23762.250000,5.013001e+15,5.011000e+15,27.000000,5.000000
max,31683.000000,5.013001e+15,5.013001e+15,99.000000,413.000000


In [27]:
test.describe()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
count,7920.000000,7.920000e+03,7.920000e+03,7920.000000
mean,3959.500000,4.637194e+15,3.807972e+15,31.822980
std,2286.451399,8.524477e+14,1.321640e+15,16.923551
min,0.000000,1.111000e+15,1.111000e+15,0.000000
25%,1979.750000,4.579000e+15,2.826000e+15,27.000000
50%,3959.500000,5.011000e+15,4.159001e+15,27.000000
75%,5939.250000,5.013001e+15,5.011000e+15,27.000000
max,7919.000000,5.013001e+15,5.013001e+15,99.000000


In [28]:
#송하인_고유식별번호
print('train: ',len(np.unique(train.송하인_격자공간고유번호)))
print('test: ',len(np.unique(test.송하인_격자공간고유번호)))
print('train+test: ',len(np.unique(np.concatenate((train.송하인_격자공간고유번호,test.송하인_격자공간고유번호)))))

train:  4229
test:  1812
train+test:  4808


In [29]:
#수하인_고유식별번호
print('train: ',len(np.unique(train.수하인_격자공간고유번호)))
print('test: ',len(np.unique(test.수하인_격자공간고유번호)))
print('train+test: ',len(np.unique(np.concatenate((train.수하인_격자공간고유번호,test.수하인_격자공간고유번호)))))

train:  26875
test:  7453
train+test:  32712


In [30]:
#물품 카테고리
print('train: ',len(np.unique(train.물품_카테고리)))
print('test: ',len(np.unique(test.물품_카테고리)))
print('train+test: ',len(np.unique(np.concatenate((train.물품_카테고리,test.물품_카테고리)))))

train:  100
test:  100
train+test:  100


#### feature 3개 모두 범주형(Categorical) 변수
송하인,수하인은 정해진 범위 없음 = 테스트 데이터에서 발생한 값이 모두 학습 데이터에 포함되지 않음
일반적으로 테스트 데이터는 모델 학습에 활용되지 않으므로 송하인, 수하인을 인코딩한 값을 활용할 경우, 성능이 좋더라도 실제로 활용할 수 없는 모델이 됨

# 데이터 시각화

# 모델링

In [34]:
from lightgbm import LGBMRegressor

X_train = train.drop('운송장_건수',axis = 1)
y_train = train['운송장_건수']

#모델 정의
model = LGBMRegressor()

In [35]:
model.fit(X_train,y_train)

LGBMRegressor()

# 모델 예측

In [36]:
pred = model.predict(test)

In [38]:
submission['운송장_건수']=pred

In [39]:
submission.head()

,index,운송장_건수
0,0,5.582915
1,1,2.267060
2,2,6.232051
3,3,4.599310
4,4,4.137938


In [40]:
submission.to_csv('submission-lightgbm.csv',index = False)